In [1]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

API_KEY = ""
RESOURCE_ENDPOINT = "" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
#Note: The openai-python library support for Azure OpenAI is in preview.
openai.api_version = "2023-03-15-preview"



### Data generation (Skip this step if data is already generated )

In [9]:
user_message = ""
def generate_titles():
    user_message =f""" 
    generate 100 titles of customer support articles in your industry, focusing on the areas of payroll and HR that support agent can look up to answer questions from customers who are employees with payroll managed by ADP.
    Output data into a list 
 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support agent for ADP company"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

titles = generate_titles()

In [10]:
titles = titles.split("\n")

In [12]:
titles[:10]

['1. Understanding Your Paycheck: A Guide for ADP Payroll Customers ',
 '2. How to Set Up Direct Deposit with ADP ',
 '3. ADP Payroll: Everything You Need to Know ',
 '4. Understanding Your W-2: A Guide for ADP Payroll Customers ',
 '5. How to Access Your ADP Payroll Account Online ',
 '6. Updating Your Personal Information in ADP Payroll ',
 '7. ADP Payroll: Troubleshooting Common Errors ',
 '8. Understanding Your ADP Pay Stub: What You Need to Know ',
 '9. ADP Payroll Taxes: What You Need to Know ',
 '10. How to Change Your Tax Withholdings with ADP Payroll ']

In [13]:
import time
def generate_article(title):
    user_message =f""" 
    given this title of an article "{title}" that support agents can look up to answer questions from customers who are employees with payroll managed by ADP.
    Generate detail content of the article 
 
"""
    i=0
    while i<5:
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a customer support agent for ADP company"},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            i+=1
            time.sleep(5)
            

contents =[]
for title in titles:
    contents.append(generate_article(title))

In [15]:
import os
folder_path ="../../../data/agent_assistant"
os.makedirs(folder_path, exist_ok=True)
i=0
articles ={}
for title, content in zip(titles, contents):
    file_name = f"article_{i}"
    file_content = title+"\n\n"+content

    with open(os.path.join(folder_path,file_name), 'w') as file:
        file.write(file_content)

    articles[file_name]= file_content
    i+=1



#Generate embeddings

In [27]:
openai.api_version = "2022-12-01"

articles_emb = {article[0]:get_embedding(article[1], engine = 'text-embedding-ada-002')for article in list(articles.items())}

### Persist data

In [32]:
import json
with open(os.path.join(folder_path,"articles_emb.json"), "w") as fp:
    json.dump(articles_emb,fp) 

with open(os.path.join(folder_path,"articles.json"), "w") as fp:
    json.dump(articles,fp) 


## Assistant Design

### Load data

In [3]:
import json
import os
folder_path ="../../../data/agent_assistant"
with open(os.path.join(folder_path, "articles_emb.json"), "r") as file:
    articles_emb = json.load(file)
with open(os.path.join(folder_path, "articles.json"), "r") as file:
    articles = json.load(file)

### 1. Tool to generate conversation

In [4]:
openai.api_version = "2023-03-15-preview"

user_message = ""
def generate_conversation():
    user_message =f""" 
    Generate a phone conversation between an customer's employee and an ADP support agent in the area of HR and Payroll. The employee has question and the support agent tried to identity the problems and 
    and take time to use different search tool to come up with answer to the employee.
    Your response:
 
"""
    system_message = """
    You are a customer support agent for ADP company. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

conversation =generate_conversation()

In [5]:
conversation

'Support Agent: "Hello, thank you for calling ADP support. This is Jessica. How may I assist you today?"\n\nEmployee: "Hi, this is Hannah from ABC Inc. I have a question about my paycheck."\n\nSupport Agent: "Sure, I\'ll be happy to help you with that. Can you please confirm your name and employee ID?"\n\nEmployee: "Yes, my name is Hannah and my employee ID is 1234."\n\nSupport Agent: "Great, thank you. What is your question about your paycheck?"\n\nEmployee: "I received a paycheck but my hours seem incorrect. What should I do?"\n\nSupport Agent: "I\'m sorry to hear that. Can you please tell me the hours you worked and what was listed on the paycheck?"\n\nEmployee: "I worked 40 hours but the paycheck only shows 32 hours."\n\nSupport Agent: "I see. Let me check your account and see what I can find. Would you mind holding for a moment while I search for this information?"\n\nEmployee: "No problem. Thank you."\n\nSupport Agent: "Thank you for your patience. Based on my research, it seems 

### 2. Tool to extract problems from conversation. 

In [24]:
openai.api_version = "2023-03-15-preview"

user_message = ""
def extract_problems(conversation):
    user_message =f""" 
    Follow this on going conversation below and extract problems that each party may need help with and formulate the search query to the knowledge base search tool.
    <<conversattion>>
    {conversation}
    <<conversattion>>
    Output your response in JSON format {{"problem":"summary of problem", "search_query":"content of search query"}}
    There can be more than 1 problem(s)
    Output just JSON, nothing else.
    Your response:
 
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    You are given access to knowledge base search tool to find knowledge needed to find answer to questions. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

problems=extract_problems(conversation)
problems

'[\n  {\n    "problem": "Employee\'s paycheck has incorrect hours",\n    "search_query": "How to correct inaccurate hours on paycheck"\n  }\n]'

### 3. Tool to find article given a problem

In [37]:
  import numpy as np  
openai.api_version = "2022-12-01"

def find_article(question):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    max_similarity = float('-inf')  
    best_file_name = None  
      
    # Compute cosine similarity between input vector and each label vector  
    for file_name, vector in articles_emb.items():  
        cosine_similarity = np.dot(input_vector, vector) / (np.linalg.norm(input_vector) * np.linalg.norm(vector))  
          
        # Update max_similarity and max_label if necessary  
        if cosine_similarity > max_similarity:  
            max_similarity = cosine_similarity  
            best_file_name = file_name  
    content = articles[best_file_name]
    return best_file_name, content
question = "When do I receive my W2 form?"
find_article(question)

('article_3',
 "4. Understanding Your W-2: A Guide for ADP Payroll Customers \n\nIf you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.\n\nTo help you understand the information displayed on your W-2 form, we've created this guide:\n\n1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.\n\n2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.\n\n3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Security wage base, which change

### 4. Tool to answer question from the given problem

In [52]:
openai.api_version = "2023-03-15-preview"

def answer_assist(question):

    article = find_article(question)
    print("article ", article)

    user_message =f""" 
    Question:{question}

    <<knowledge articles>>
    name of article: {article[0]}
    content:
    {article[1]}
    <<knowledge articles>>
    Your response:
 
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    Given the question or problem statement and the knowledge article you have, give the answer.
    Rely solely to the guidance from the article.If the knowlege article is not relavant to the question, say you don't know. Do not make up your answer. 
    Cite the name of the knowledge article as source for your answer.
    Format:
    Answer: your answer to the question
    Sources: [source1, source2]
    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content'], article

question = "Why is my paycheck amount lower than expected?"
answer, article = answer_assist(question)
print(answer)
print("---------------content-----------------")

print(article[1])

article  ('article_0', "1. Understanding Your Paycheck: A Guide for ADP Payroll Customers \n\n1. Understanding Your Paycheck: A Guide for ADP Payroll Customers\n\nAs an ADP Payroll customer, you may receive your paycheck via direct deposit or a physical check. It is important to understand the different sections of your paycheck and what they represent. This guide will help you understand your paycheck and answer any questions you may have.\n\nGross Pay\n\nThe first section of your paycheck will show your gross pay. This is the amount of money you earned before any taxes, deductions, or contributions were taken out. Your gross pay may include regular pay, overtime pay, holiday pay, or any other additional compensation.\n\nTaxes\n\nThe next section of your paycheck will show the different taxes you are required to pay. This may include federal income tax, state income tax, Social Security tax, and Medicare tax. The amount of taxes you pay will depend on your income, your tax exemptions,

### 5. Put tools all together

In [55]:
def demo():
    conversation =generate_conversation()
    print(f"Conversation {conversation}")
    problems=extract_problems(conversation)
    problems=json.loads(problems)
    for problem in problems:
        print(f"Problem: {problem['problem']}")
        answer, article = answer_assist(problem['search_query'])
        print(answer)
        print("---------------content-----------------")

        print(article[1])
        

    

In [56]:
demo()

Conversation ADP Support Agent: Thank you for calling ADP, this is John speaking, how can I assist you today?

Customer's Employee: Hi John, I have a question about my paycheck. It seems like there was a mistake and I am not sure how to fix it.

ADP Support Agent: I'm here to help. Can you please tell me your name and the name of your company?

Customer's Employee: Sure, my name is Jane Smith and I work for ABC Company.

ADP Support Agent: Thank you, Jane. Can you give me a little more information about the mistake on your paycheck?

Customer's Employee: Yes, it looks like my vacation time wasn't calculated properly and I didn't receive the correct amount of pay.

ADP Support Agent: I'm sorry to hear that. Let me take a look at your account and see if I can identify the issue. Can you please hold for a moment?

(Customer's Employee holds for a few minutes while the ADP Support Agent searches through the system)

ADP Support Agent: Thank you for waiting, Jane. After reviewing your accou